# Práctica 4 - Visión por Computador

### Autores

- Pablo Segura López
- Kilian Armas Pérez

### Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO

## Tarea - Detección de Matrículas en Vídeo con YOLO

In [11]:
import numpy as np
#import pytesseract
#from pytesseract import Output
import easyocr
import csv

# Si la ruta de Tesseract no está en el PATH, ruta al ejecutable
#pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
reader = easyocr.Reader(['en', 'es'], gpu=True)

# Variable booleana que indica si quieres guardar en memoria (False) o visualizar en pantalla (True)
display = False

# Carga de los modelos
model = YOLO('YOLO/yolo11n.pt')
license_plate_model = YOLO('YOLO/best.pt')

# Etiqueta de las distintas clases y cuenta sus ocurrencias
classNames = ["person", "bicycle", "car", "motorbike", "","bus","", "truck"]
unique_ocurrences = [set(), set(), set(), set(), set(), set(), set(), set()]

# Captura desde la webcam
vid = cv2.VideoCapture("YOLO/C0142.MP4")
# Escritura del vídeo en disco
if not display:
    frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_video = cv2.VideoWriter('YOLO/resultados.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))

possible_licenses_confs = {}

# Inicialización de CSV
with open('resultados.csv', mode='w', newline='') as csv_file:
    fieldnames = [
        'fotograma', 'tipo_objeto', 'confianza', 'identificador_tracking', 
        'x1', 'y1', 'x2', 'y2', 'matrícula', 'confianza_matrícula', 
        'mx1', 'my1', 'mx2', 'my2', 'texto_matrícula'
    ]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    frame_number = 0  # Contador de fotogramas
  
    while(True):      
        # fotograma a fotograma
        ret, img = vid.read()
    
        # si hay imagen válida
        if ret:  
            # Seguimiento, con persistencia entre fotogramas
            results = model.track(img, persist=True, classes = [0,1,2,3,5,7])
            
            # Para cada detección
            for r in results:
                boxes = r.boxes

                for box in boxes:
                    # Contenedor
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                    #Etiqueta de seguimiento
                    if box.id is not None:
                        track_id = str(int(box.id[0].tolist()))
                    else:
                        track_id = ''
                    
                    # Confianza
                    confidence = math.ceil((box.conf[0]*100))/100
                    print("Confianza --->",confidence)

                    # Clase
                    cls = int(box.cls[0])
                    unique_ocurrences[cls].add(track_id)
                    print("Clase -->", classNames[cls])
                    # Matriz de datos inicial para el CSV
                    data = {
                        'fotograma': frame_number,
                        'tipo_objeto': classNames[cls],
                        'confianza': confidence,
                        'identificador_tracking': track_id,
                        'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                        'matrícula': False,
                        'confianza_matrícula': '',
                        'mx1': '', 'my1': '', 'mx2': '', 'my2': '',
                        'texto_matrícula': ''
                    }

                    if cls >= 2:
                        # Recorta el contenedor del coche
                        car_crop = img[y1:y2, x1:x2]

                        # Detecta la matrícula en el contenedor del coche
                        license_plate_results = license_plate_model.predict(car_crop)
                        # Procesa cada detección en el contenedor del coche
                        for lp_result in license_plate_results:
                            lp_boxes = lp_result.boxes
                            for lp_box in lp_boxes:
                                lp_x1, lp_y1, lp_x2, lp_y2 = lp_box.xyxy[0]
                                lp_x1, lp_y1, lp_x2, lp_y2 = int(lp_x1), int(lp_y1), int(lp_x2), int(lp_y2)
                                if not track_id in possible_licenses_confs:
                                    possible_licenses_confs[track_id] = {"Not Found": 0}
                                license_plate_text = reader.readtext(car_crop[lp_y1:lp_y2, lp_x1:lp_x2])
                                if license_plate_text:
                                    _, predicted_license, confidence = license_plate_text[0]
                                else:
                                    predicted_license = "Not Found"
                                    confidence = 0
                                #predicted_license = pytesseract.image_to_string(car_crop[lp_y1:lp_y2, lp_x1:lp_x2])
                                #predicted_license_data = pytesseract.image_to_data(car_crop[lp_y1:lp_y2, lp_x1:lp_x2], output_type=Output.DICT)
                                #confidences = predicted_license_data['conf']
                                #confidences = [int(conf) for conf in confidences if int(conf) != -1]
                                if len(predicted_license) >= 3:
                                    possible_licenses_confs[track_id][predicted_license] = confidence if confidence else 0
                                else:
                                    possible_licenses_confs[track_id][predicted_license] = 0
                                most_possible_text = max(possible_licenses_confs[track_id], key=possible_licenses_confs[track_id].get)
                                # Ajustar coordenadas al área de imagen completa
                                lp_x1 += x1
                                lp_x2 += x1
                                lp_y1 += y1
                                lp_y2 += y1
                                data.update({
                                    'matrícula': True,
                                    'confianza_matrícula': possible_licenses_confs[track_id][predicted_license],
                                    'mx1': lp_x1, 'my1': lp_y1,
                                    'mx2': lp_x2, 'my2': lp_y2,
                                    'texto_matrícula': most_possible_text
                                })

                                # Dibuja el contenedor de la matrícula sobre el coche
                                cv2.rectangle(img, (lp_x1, lp_y1), (lp_x2, lp_y2), (0, 255, 0), 2)
                                cv2.putText(img,
                                            f"Matricula: {most_possible_text}",
                                            (lp_x1, lp_y1 - 10),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Convierte identificador numérico de clase a un color RGB
                    escala = int((cls / len(classNames)) * 255 * 3)
                    if escala >= 255*2:
                        R = 255
                        G = 255
                        B = escala - 255*2
                    else:
                        if escala >= 255:
                            R = 255
                            G = escala - 255
                            B = 0
                        else:
                            R = escala
                            G = 0
                            B = 0
                    # Dibuja el contenedor y clase
                    cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                    cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)
                    # Escribe en el CSV
                    if not display:
                        writer.writerow(data)
            # Muestra fotograma
            if display:
                cv2.imshow('Vid', img)
            else:
                # Muestra fotograma
                output_video.write(img)
            # Detenemos pulsado ESC
            if cv2.waitKey(20) == 27:
                break
        else:
            break
  
# Libera el objeto de captura
vid.release()
if display:
    cv2.destroyAllWindows()
else:
    output_video.release()  # Libera el archivo de video



print("Resultados: ")
for i in range(len(classNames)):
    if i == 4 or i == 6:
        continue
    print(f"{len(unique_ocurrences[i])} {classNames[i]} encontradas en el vídeo")


0: 384x640 1 car, 1 bus, 10.5ms
Speed: 1.0ms preprocess, 10.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.94
Clase --> bus

0: 352x416 1 License_Plate, 8.1ms
Speed: 0.0ms preprocess, 8.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)
Confianza ---> 0.85
Clase --> car

0: 320x416 (no detections), 10.6ms
Speed: 1.0ms preprocess, 10.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 416)

0: 384x640 1 car, 1 bus, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.94
Clase --> bus

0: 352x416 1 License_Plate, 14.5ms
Speed: 1.0ms preprocess, 14.5ms inference, 2.5ms postprocess per image at shape (1, 3, 352, 416)
Confianza ---> 0.82
Clase --> car

0: 352x416 (no detections), 9.8ms
Speed: 1.0ms preprocess, 9.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 416)

0: 384x640 1 car, 1 bus, 7.1ms
Speed: 2.0ms preprocess, 7.1ms inference, 1.